In [3]:
!pip install -q datasets transformers peft accelerate bitsandbytes

In [4]:
#Load Dataset
from datasets import load_dataset

print("Loading dataset...")
dataset = load_dataset("tolu07/Mental_Health_FAQ")
print("Dataset loaded!")


Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset loaded!


In [5]:
dataset['train'][0]

{'Question_ID': 1590140,
 'Questions': 'What does it mean to have a mental illness?',
 'Answers': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condit

In [6]:
print(dataset['train'].column_names)


['Question_ID', 'Questions', 'Answers']


In [7]:
#Format the dataset
def format_example(example):
    instruction = example.get('Questions', "")
    response = example.get('Answers', "")
    return {
        'text': f"### Question:\n{instruction}\n\n### Answer:\n{response}"
    }


In [8]:
formatted_dataset = dataset['train'].map(
    format_example,
    remove_columns=dataset['train'].column_names,  # keeps only 'text'
    desc="Formatting examples"
)


In [9]:
#Split dataset into train and validation (90% train, 10% val)
split_dataset = formatted_dataset.train_test_split(test_size=0.1)

train_dataset = split_dataset['train']
val_dataset = split_dataset['test']


In [10]:
#Load the Tokenizer
from transformers import AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Some small models don't have a padding token, so we set it explicitly
tokenizer.pad_token = tokenizer.eos_token


In [11]:
#Tokenize the Text

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [12]:
#Load the Model
from transformers import AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

In [13]:
#Apply Lora
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

#Prepare for k-bit training (must come before LoRA config)
model = prepare_model_for_kbit_training(model)

#LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

#Wrap the model with LoRA
model = get_peft_model(model, lora_config)

#Show how many parameters are trainable
model.print_trainable_parameters()


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [14]:
#Set up Training Arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./tinyllama-mentalhealth",
    per_device_train_batch_size=1,        # Adjust if you get memory errors (try 2)
    gradient_accumulation_steps=4,        # Simulates larger batch size
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    fp16=False,
    bf16=False,
    optim="adamw_torch",                  # Use adamw optimizer for CPU
    report_to="none",
    push_to_hub=False
)


In [15]:
#Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Because we're training a causal language model, not BERT
)


In [17]:
#Train the Model
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments

#Define compute_metrics to calculate accuracy during evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return {"accuracy": accuracy_score(labels, predictions)}

#Training arguments
training_args = TrainingArguments(
    output_dir="./tinyllama-mentalhealth",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=10,
    save_total_limit=2,
    fp16=False,
    bf16=False,
    optim="adamw_torch",
    report_to="none",
    push_to_hub=False
)


#Create Trainer with validation dataset and metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-17-4171653212>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,1.895200
20,1.815500
30,1.822400
40,1.758900


Step,Training Loss
10,1.895200
20,1.815500
30,1.822400
40,1.758900
50,1.743200
60,1.725000


TrainOutput(global_step=66, training_loss=1.7863590067083186, metrics={'train_runtime': 12055.8737, 'train_samples_per_second': 0.022, 'train_steps_per_second': 0.005, 'total_flos': 839911338934272.0, 'train_loss': 1.7863590067083186, 'epoch': 3.0})

In [18]:
#Save the Model
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/chat_template.jinja',
 './final_model/tokenizer.model',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')

In [19]:
#Load the Model and Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load your fine-tuned model
model = AutoModelForCausalLM.from_pretrained("./final_model")
tokenizer = AutoTokenizer.from_pretrained("./final_model")


In [20]:
#Testing the Model
# Format the input like your training template
question = "What can I do if I feel anxious all the time?"
prompt = f"### Question:\n{question}\n\n### Answer:"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate a response
outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id  # prevent warning
)

# Decode and print
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


### Question:
What can I do if I feel anxious all the time?

### Answer:
It’s normal for some people to feel anxious at times. Anxiety can be uncomfortable, disruptive, and can affect your daily life. The good news is that anxiety is a common experience, and it’s not necessarily a sign of something wrong with you. 

If you’re feeling anxious, it’s important to talk to your healthcare provider. It’s okay to have a conversation about anxiety with a healthcare provider, and they can provide support, guidance, and help you find the resources you need. 

If you’re feeling anxious all the time, there are things you can try to help manage your anxiety. These are called strategies. Some


In [21]:
#Zip the model folder
!zip -r final_model.zip final_model


  adding: final_model/ (stored 0%)
  adding: final_model/chat_template.jinja (deflated 60%)
  adding: final_model/training_args.bin (deflated 52%)
  adding: final_model/special_tokens_map.json (deflated 73%)
  adding: final_model/tokenizer.model (deflated 55%)
  adding: final_model/adapter_config.json (deflated 54%)
  adding: final_model/tokenizer.json (deflated 85%)
  adding: final_model/tokenizer_config.json (deflated 69%)
  adding: final_model/README.md (deflated 66%)
  adding: final_model/adapter_model.safetensors (deflated 8%)


In [22]:
#Download the Model
from google.colab import files
files.download("final_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>